In [32]:
import langgraph.graph as lg
from langgraph.graph import END, StateGraph, START
import requests
import subprocess
import time
import os
from typing import TypedDict

In [33]:
GROQ_API_KEY = "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"

In [34]:
def call_groq_llm(prompt):
    url = "https://api.groq.com/v1/chat/completions"
    headers = {"Authorization": f"Bearer {GROQ_API_KEY}", "Content-Type": "application/json"}
    data = {
        "model": "llama3-70b",  # Using Llama 3 from Groq
        "messages": [{"role": "system", "content": "You are an AI coding assistant."},
                     {"role": "user", "content": prompt}]
    }
    response = requests.post(url, json=data, headers=headers)
    return response.json()["choices"][0]["message"]["content"]

In [35]:
class WorkflowState(TypedDict):
    js_code: str  # Stores JavaScript code input
    py_code: str  # Stores Python code output

In [36]:
js_file_path = r"C:\Users\shant\Desktop\Shantanu Stuff\Study stuff\Lang graph\test.js"

In [37]:
# Agent 1: Function to read the JavaScript file
def read_js_code(file_path: str) -> dict:
    print("\n🔹 Reading JavaScript Code...")
    with open(file_path, "r") as f:
        js_code = f.read()
    print(f"\n📜 JavaScript Code:\n{js_code}")
    return {"js_code": js_code}

# ✅ Call the function with the file path variable
output = read_js_code(js_file_path)

# ✅ Print the output dictionary
print("\n🔹 Final Output:", output)



🔹 Reading JavaScript Code...

📜 JavaScript Code:
// Simple JavaScript function
function fibonacci(n) {
    if (n <= 1) return n;
    return fibonacci(n - 1) + fibonacci(n - 2);
}

console.log(fibonacci(10)); // Expected Output: 55


🔹 Final Output: {'js_code': '// Simple JavaScript function\nfunction fibonacci(n) {\n    if (n <= 1) return n;\n    return fibonacci(n - 1) + fibonacci(n - 2);\n}\n\nconsole.log(fibonacci(10)); // Expected Output: 55\n'}


In [38]:

# Agent 2: Convert JavaScript to Python using Groq’s Llama 3
def convert_to_python(state: WorkflowState) -> dict:
    print("\n🔹 Converting JavaScript to Python using Llama 3...")
    js_code = state["js_code"]
    py_code = call_groq_llm(f"Convert this JavaScript code to Python:\n\n{js_code}")
    print(f"\n🐍 Converted Python Code:\n{py_code}")
    return {"py_code": py_code}

In [39]:
# ✅ Agent 3: Validate Python Code Execution
def validate_python_code(state: WorkflowState, attempt=1) -> str:
    print(f"\n🔹 Running Python Code (Attempt {attempt})...")

    # Save converted Python code to a file
    with open("test.py", "w") as f:
        f.write(state["py_code"])

    try:
        # Execute the Python script
        output = subprocess.check_output(["python", "test.py"], stderr=subprocess.STDOUT, text=True)
        print(f"\n✅ Output: {output.strip()}")

        if "55" in output:  # ✅ Expected output for Fibonacci(10) (Adjust if needed)
            return "✅ Success: Code executed correctly!"
        else:
            if attempt >= 5:
                return "🚨 Human intervention needed for the code."
            time.sleep(1)  # Wait before retrying
            return validate_python_code(state, attempt + 1)

    except subprocess.CalledProcessError:
        print("\n❌ Code execution failed.")
        if attempt >= 5:
            return "🚨 Human intervention needed for the code."
        time.sleep(1)  # Wait before retrying
        return validate_python_code(state, attempt + 1)

In [40]:


# ✅ Define Workflow State Schema
class WorkflowState(TypedDict):
    js_code: str  # Stores JavaScript code input
    py_code: str  # Stores Python code output

# ✅ Initialize StateGraph with Schema
workflow = lg.StateGraph(WorkflowState)

print("✅ LangGraph workflow initialized successfully!")


✅ LangGraph workflow initialized successfully!


In [41]:
# ✅ Initialize LangGraph Workflow
workflow = lg.StateGraph(WorkflowState)

In [42]:
# ✅ Add Nodes to Workflow
workflow.add_node("read_js", read_js_code)
workflow.add_node("convert_python", convert_to_python)
workflow.add_node("validate_python", validate_python_code)

In [43]:
# ✅ Define Workflow Execution Order
workflow.add_edge("read_js", "convert_python")
workflow.add_edge("convert_python", "validate_python")

In [44]:
# Set Entry Point
workflow.set_entry_point("read_js")

In [45]:
# ✅ Compile the Workflow Before Running
executor = workflow.compile()

In [ ]:
# ✅ Run the LangGraph Workflow
js_file_path = "test.js"  # Ensure you have a JavaScript file named test.js
output = executor.invoke({"read_js": js_file_path})

print("\n🔹 Final Output:", output)


🔹 Reading JavaScript Code...


TypeError: unhashable type: 'dict'